In [1]:
from langchain import LLMChain, PromptTemplate
from langchain.llms import LLAMA
from langchain.agents import create_sql_agent, SQLAgent
from sqlalchemy import create_engine, text
import os

ModuleNotFoundError: No module named 'pytorch'

In [ ]:
DATABASE_URL = os.environ.get("MONGOURI")
engine = create_engine(DATABASE_URL)


In [ ]:
prompt_template = """
You are an assistant that retrieves information from a database. 
When asked about occupancy or crowd monitoring, use the provided database connection to fetch relevant details.
Consider the following:
- Time: {time}
- Place: {place}

Query: {query}
"""

In [ ]:
llm = LLAMA(model_name='redbackbot')
llm_chain = LLMChain(
    llm=llm,
    prompt_template=PromptTemplate(template=prompt_template),
)

In [ ]:
sql_agent = create_sql_agent(llm_chain, engine)

In [ ]:
def get_dynamic_info(time: str, place: str, query_template: str):
    try:
        # Format the query with the provided time and place
        query = query_template.format(time=time, place=place)
        # Use the agent to run the SQL query
        result = sql_agent.run(query)
        return result
    except Exception as e:
        return str(e)

In [ ]:
# Example function usage: Get current occupancy info based on time and place
def get_occupancy_info(time: str, place: str):
    query_template = "SELECT COUNT(*) AS current_occupancy FROM occupancy_table WHERE status = 'occupied' AND time = '{time}' AND place = '{place}';"
    return get_dynamic_info(time, place, query_template)

# Example function usage: Get crowd monitoring info based on time and place
def get_crowd_info(time: str, place: str):
    query_template = "SELECT area, COUNT(*) AS crowd_count FROM crowd_monitoring_table WHERE time = '{time}' AND place = '{place}' GROUP BY area;"
    return get_dynamic_info(time, place, query_template)

In [ ]:
time_input = "2024-09-01 12:00:00"
place_input = "Main Hall"

occupancy_info = get_occupancy_info(time_input, place_input)
print(f"Occupancy Information for {place_input} at {time_input}: {occupancy_info}")

crowd_info = get_crowd_info(time_input, place_input)
print(f"Crowd Monitoring Information for {place_input} at {time_input}: {crowd_info}")